In [ ]:
from ase.io import read
from gpaw import GPAW, FermiDirac, PW
from ase.visualize import view
import numpy as np
import pandas as pd
from dftd4.ase import DFTD4
from dftd3.ase import DFTD3
from ase.build import fcc100

gp = GPAW(mode=PW(500),
          xc='PBE',
          kpts=(3, 3, 1),
          occupations=FermiDirac(0.1),
          txt='gpaw.txt',)


atoms = fcc100('Cu', size=(1,1,1),vacuum=5)

view(atoms)
#atoms.calc = gp
atoms.calc = SumCalculator([gp, DFTD4(method='PBE', damping='d3bj')])
print('D4: ',atoms.get_potential_energy())
atoms.calc= SumCalculator([gp, DFTD3(method='PBE',damping = 'd3bj')])    
print('D3: ',atoms.get_potential_energy())

D4:  -2.4850464555970335
D3:  -2.471274320241147


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Load the DataFrame
df = pd.read_excel('ecut_convergence_results_fixed.xlsx')

# Get unique metal-functional combinations
combinations = df[['Metal', 'Functional']].drop_duplicates().reset_index(drop=True)

# Plot setup
fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(12, 18), sharex=True)
axes = axes.flatten()
sns.set(style='whitegrid')

# Plot each metal-functional combination
for i, (metal, functional) in combinations.iterrows():
    subset = df[(df['Metal'] == metal) & (df['Functional'] == functional)]
    ax = axes[i]
    sns.lineplot(data=subset, x='Ecut (eV)', y='Energy (eV)', marker='o', ax=ax)
    ax.set_title(f'{metal} - {functional}')
    ax.set_xlabel('Ecut (eV)', fontweight='bold')
    ax.set_ylabel('Energy (eV)', fontweight='bold')

# Adjust layout
plt.tight_layout()
subplot_path = '/mnt/data/All_Metals_Functionals_Ecut_Energy_Subplots.png'
plt.savefig(subplot_path)
plt.close()

subplot_path


In [ ]:
from dftd4.ase import DFTD4
from dftd3.ase import DFTD3
from gpaw import GPAW, PW, FermiDirac
from ase.build import bulk
from ase.visualize import view

cu_crystal = bulk('Cu', 'fcc')  # Create a Cu(100) surface
view(cu_crystal)  # Visualize the crystal structure
ecut_value = 400
kpnts_mesh = (3, 3, 3)  # k-point mesh for the calculation

calc = DFTD3(method="PBE", damping="d3bj", calculator=GPAW(mode=PW(ecut_value),
                                                        xc='PBE',
                                                        txt='Cu_D3_lc.txt', # Unique txt file for each scaling
                                                        kpts={'size': kpnts_mesh},
                                                        occupations=FermiDirac(0.05)))

cu_crystal.calc = calc  # Assign the calculator to the crystal
energy = cu_crystal.get_potential_energy()  # Calculate the potential energy
print(f'Potential energy of Cu(100) surface with DFT-D3: {energy:.3f} eV')



Potential energy of Cu(100) surface with DFT-D3: -0.514 eV


In [ ]:
from dftd4.ase import DFTD4
from gpaw import GPAW, PW, FermiDirac
from ase.build import bulk
from ase.visualize import view

cu_crystal = bulk('Cu', 'fcc')  # Create a Cu(100) surface
view(cu_crystal)  # Visualize the crystal structure
ecut_value = 400
kpnts_mesh = (3, 3, 3)  # k-point mesh for the calculation

calc = DFTD4(mode=PW(ecut_value),xc='PBE',txt='Cu_D4_lc.txt',kpts={'size': kpnts_mesh},occupations=FermiDirac(0.05),
                        method="PBE", damping="d3bj")

cu_crystal.calc = calc  # Assign the calculator to the crystal
energy = cu_crystal.get_potential_energy()  # Calculate the potential energy
print(f'Potential energy of Cu(100) surface with DFT-D4: {energy:.3f} eV')



Potential energy of Cu(100) surface with DFT-D4: -0.563 eV


In [12]:
from gpaw import GPAW, PW, FermiDirac
from ase.build import bulk
from ase.visualize import view

cu_crystal = bulk('Cu', 'fcc')  # Create a Cu(100) surface
view(cu_crystal)  # Visualize the crystal structure
ecut_value = 400
kpnts_mesh = (3, 3, 3)  # k-point mesh for the calculation

calc = GPAW(mode=PW(ecut_value),
            xc='PBE',
            txt='Cu_lc.txt',  # Unique txt file for each scaling
            kpts={'size': kpnts_mesh},
            occupations=FermiDirac(0.05))



cu_crystal.calc = calc  # Assign the calculator to the crystal
energy = cu_crystal.get_potential_energy()  # Calculate the potential energy
print(f'Potential energy of Cu(100) surface just PBE: {energy:.3f} eV')



Potential energy of Cu(100) surface just PBE: -3.883 eV


In [14]:
# Import necessary libraries
from gpaw import GPAW, PW, FermiDirac
from ase.build import bulk
from ase.eos import EquationOfState
from ase.units import GPa
import matplotlib.pyplot as plt
import numpy as np

# Import DFT-D3 and DFT-D4 calculators
from dftd3.ase import DFTD3
from dftd4.ase import DFTD4

# Define common parameters
ecut_value = 400  # eV
kpnts_mesh = (3, 3, 3) # k-point mesh for the calculation
smearing_width = 0.05 # Fermi-Dirac smearing width

# Function to create a GPAW calculator
def get_gpaw_calculator(txt_file_name):
    """
    Returns a GPAW calculator with specified parameters.
    """
    calc = GPAW(mode=PW(ecut_value),
                xc='PBE',
                txt=txt_file_name,
                kpts={'size': kpnts_mesh},
                occupations=FermiDirac(smearing_width))
    return calc

# --- PBE Only Calculation ---
print("--- Starting PBE-only Lattice Constant Optimization ---")

# Define a range of lattice constants to scan
# Experimental lattice constant for Cu is around 3.615 Å
# We'll scan a range around this value
lattice_constants_pbe = np.linspace(3.5, 3.7, 7) # Scan 7 points between 3.5 and 3.7 Angstroms
energies_pbe = []
volumes_pbe = []

for a in lattice_constants_pbe:
    cu_crystal = bulk('Cu', 'fcc', a=a)
    calc_pbe = get_gpaw_calculator(f'Cu_PBE_lc_{a:.3f}.txt')
    cu_crystal.calc = calc_pbe
    energy = cu_crystal.get_potential_energy()
    energies_pbe.append(energy)
    volumes_pbe.append(cu_crystal.get_volume())
    print(f'PBE: Lattice Constant = {a:.3f} Å, Energy = {energy:.3f} eV, Volume = {cu_crystal.get_volume():.3f} Å^3')

# Fit Equation of State for PBE
eos_pbe = EquationOfState(volumes_pbe, energies_pbe)
v0_pbe, e0_pbe, B_pbe = eos_pbe.fit()
a0_pbe = (v0_pbe / 4)**(1/3) # For FCC, 4 atoms per unit cell

print(f"\nPBE Results:")
print(f"Equilibrium Volume (V0): {v0_pbe:.3f} Å^3")
print(f"Equilibrium Energy (E0): {e0_pbe:.3f} eV")
print(f"Bulk Modulus (B): {B_pbe / GPa:.3f} GPa")
print(f"Equilibrium Lattice Constant (a0): {a0_pbe:.3f} Å")

# --- PBE + DFT-D3 Calculation ---
print("\n--- Starting PBE+DFTD3 Lattice Constant Optimization ---")

lattice_constants_d3 = np.linspace(3.5, 3.7, 7)
energies_d3 = []
volumes_d3 = []

for a in lattice_constants_d3:
    cu_crystal = bulk('Cu', 'fcc', a=a)
    # Correctly instantiate DFTD3: it takes a GPAW calculator as an argument
    gpaw_calc_d3 = get_gpaw_calculator(f'Cu_D3_lc_{a:.3f}.txt')
    calc_d3 = DFTD3(method="PBE", damping="d3bj", calculator=gpaw_calc_d3)
    cu_crystal.calc = calc_d3
    energy = cu_crystal.get_potential_energy()
    energies_d3.append(energy)
    volumes_d3.append(cu_crystal.get_volume())
    print(f'PBE+D3: Lattice Constant = {a:.3f} Å, Energy = {energy:.3f} eV, Volume = {cu_crystal.get_volume():.3f} Å^3')

# Fit Equation of State for PBE+DFTD3
eos_d3 = EquationOfState(volumes_d3, energies_d3)
v0_d3, e0_d3, B_d3 = eos_d3.fit()
a0_d3 = (v0_d3 / 4)**(1/3)

print(f"\nPBE+DFTD3 Results:")
print(f"Equilibrium Volume (V0): {v0_d3:.3f} Å^3")
print(f"Equilibrium Energy (E0): {e0_d3:.3f} eV")
print(f"Bulk Modulus (B): {B_d3 / GPa:.3f} GPa")
print(f"Equilibrium Lattice Constant (a0): {a0_d3:.3f} Å")

# --- PBE + DFT-D4 Calculation ---
print("\n--- Starting PBE+DFTD4 Lattice Constant Optimization ---")

lattice_constants_d4 = np.linspace(3.5, 3.7, 7)
energies_d4 = []
volumes_d4 = []

for a in lattice_constants_d4:
    cu_crystal = bulk('Cu', 'fcc', a=a)
    # Correctly instantiate DFTD4: it takes a GPAW calculator as an argument
    gpaw_calc_d4 = get_gpaw_calculator(f'Cu_D4_lc_{a:.3f}.txt')
    calc_d4 = DFTD4(method="PBE", calculator=gpaw_calc_d4) # DFTD4 'damping' is often chosen internally based on method
    cu_crystal.calc = calc_d4
    energy = cu_crystal.get_potential_energy()
    energies_d4.append(energy)
    volumes_d4.append(cu_crystal.get_volume())
    print(f'PBE+D4: Lattice Constant = {a:.3f} Å, Energy = {energy:.3f} eV, Volume = {cu_crystal.get_volume():.3f} Å^3')

# Fit Equation of State for PBE+DFTD4
eos_d4 = EquationOfState(volumes_d4, energies_d4)
v0_d4, e0_d4, B_d4 = eos_d4.fit()
a0_d4 = (v0_d4 / 4)**(1/3)

print(f"\nPBE+DFTD4 Results:")
print(f"Equilibrium Volume (V0): {v0_d4:.3f} Å^3")
print(f"Equilibrium Energy (E0): {e0_d4:.3f} eV")
print(f"Bulk Modulus (B): {B_d4 / GPa:.3f} GPa")
print(f"Equilibrium Lattice Constant (a0): {a0_d4:.3f} Å")

# --- Plotting the Results ---
plt.figure(figsize=(10, 6))

# PBE plot
volumes_plot_pbe = np.linspace(min(volumes_pbe), max(volumes_pbe), 100)
energies_plot_pbe = eos_pbe.eos(volumes_plot_pbe)
plt.plot(volumes_plot_pbe, energies_plot_pbe, label=f'PBE (a0={a0_pbe:.3f} Å)')
plt.plot(volumes_pbe, energies_pbe, 'o', markersize=6)

# PBE+DFTD3 plot
volumes_plot_d3 = np.linspace(min(volumes_d3), max(volumes_d3), 100)
energies_plot_d3 = eos_d3.eos(volumes_plot_d3)
plt.plot(volumes_plot_d3, energies_plot_d3, label=f'PBE+DFTD3 (a0={a0_d3:.3f} Å)')
plt.plot(volumes_d3, energies_d3, '^', markersize=6)

# PBE+DFTD4 plot
volumes_plot_d4 = np.linspace(min(volumes_d4), max(volumes_d4), 100)
energies_plot_d4 = eos_d4.eos(volumes_plot_d4)
plt.plot(volumes_plot_d4, energies_plot_d4, label=f'PBE+DFTD4 (a0={a0_d4:.3f} Å)')
plt.plot(volumes_d4, energies_d4, 's', markersize=6)

plt.xlabel('Volume ($\AA^3$)')
plt.ylabel('Energy (eV)')
plt.title('Energy vs. Volume for Cu FCC')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

<>:139: SyntaxWarning: invalid escape sequence '\A'
<>:139: SyntaxWarning: invalid escape sequence '\A'
/tmp/ipykernel_11132/3062693242.py:139: SyntaxWarning: invalid escape sequence '\A'
  plt.xlabel('Volume ($\AA^3$)')


--- Starting PBE-only Lattice Constant Optimization ---
PBE: Lattice Constant = 3.500 Å, Energy = -3.827 eV, Volume = 10.719 Å^3
PBE: Lattice Constant = 3.533 Å, Energy = -3.851 eV, Volume = 11.028 Å^3
PBE: Lattice Constant = 3.567 Å, Energy = -3.879 eV, Volume = 11.343 Å^3
PBE: Lattice Constant = 3.600 Å, Energy = -3.887 eV, Volume = 11.664 Å^3
PBE: Lattice Constant = 3.633 Å, Energy = -3.889 eV, Volume = 11.991 Å^3
PBE: Lattice Constant = 3.667 Å, Energy = -3.866 eV, Volume = 12.324 Å^3
PBE: Lattice Constant = 3.700 Å, Energy = -3.852 eV, Volume = 12.663 Å^3

PBE Results:
Equilibrium Volume (V0): 11.792 Å^3
Equilibrium Energy (E0): -3.887 eV
Bulk Modulus (B): 193.901 GPa
Equilibrium Lattice Constant (a0): 1.434 Å

--- Starting PBE+DFTD3 Lattice Constant Optimization ---
PBE+D3: Lattice Constant = 3.500 Å, Energy = -0.559 eV, Volume = 10.719 Å^3
PBE+D3: Lattice Constant = 3.533 Å, Energy = -0.545 eV, Volume = 11.028 Å^3
PBE+D3: Lattice Constant = 3.567 Å, Energy = -0.531 eV, Volume = 

/tmp/ipykernel_11132/3062693242.py:139: SyntaxWarning: invalid escape sequence '\A'
  plt.xlabel('Volume ($\AA^3$)')


AttributeError: 'EquationOfState' object has no attribute 'eos'

<Figure size 1000x600 with 0 Axes>

In [24]:
# Import necessary libraries
from gpaw import GPAW, PW, FermiDirac
from ase.build import bulk
from ase.calculators.mixing import SumCalculator
import numpy as np
import sys # Import sys to potentially redirect output if needed, but not doing it here

# Import DFT-D3 and DFT-D4 calculators
from dftd3.ase import DFTD3
from dftd4.ase import DFTD4

# Define common parameters
ecut_value = 400  # eV
kpnts_mesh = (3, 3, 3) # k-point mesh for the calculation
smearing_width = 0.05 # Fermi-Dirac smearing width
experimental_lc = 3.615 # Experimental lattice constant for Cu FCC in Å

# Function to create a GPAW calculator
def get_gpaw_calculator(txt_file_name):
    """
    Returns a GPAW calculator with specified parameters.
    The SCF output will go to txt_file_name.
    """
    calc = GPAW(mode=PW(ecut_value),
                xc='PBE',
                txt=txt_file_name, # Output goes to this file
                kpts={'size': kpnts_mesh},
                occupations=FermiDirac(smearing_width),
                verbose=2 # Maximize verbosity in the file
                )
    return calc

# --- Define Lattice Constants for Iteration ---
# Start 2% below experimental, end 2% above, with 5 steps
lc_start = experimental_lc * (1 - 0.04)
lc_end = experimental_lc * (1 + 0.02)
num_steps = 5
lattice_constants_to_test = np.linspace(lc_start, lc_end, num_steps)

all_results = [] # To store results for all LCs

print("--- Starting Lattice Constant Scan (Detailed SCF output in .txt files) ---")

for i, current_lc in enumerate(lattice_constants_to_test):
    print(f"Calculating for LC = {current_lc:.3f} Å...")
    
    current_step_results = {"LC": current_lc}

    # --- PBE Only Calculation ---
    cu_crystal_pbe = bulk('Cu', 'fcc', a=current_lc)
    calc_pbe = get_gpaw_calculator(f'Cu_PBE_lc_{current_lc:.3f}.txt')
    cu_crystal_pbe.calc = calc_pbe
    energy_pbe = cu_crystal_pbe.get_potential_energy()
    current_step_results["PBE_Energy"] = energy_pbe

    # --- PBE + DFT-D3 Calculation (using SumCalculator) ---
    cu_crystal_d3 = bulk('Cu', 'fcc', a=current_lc)
    gpaw_calc_d3_part = get_gpaw_calculator(f'Cu_D3_Sum_lc_{current_lc:.3f}.txt')
    dftd3_part = DFTD3(method="PBE", damping="d3bj")
    calc_d3_sum = SumCalculator([gpaw_calc_d3_part, dftd3_part])
    cu_crystal_d3.calc = calc_d3_sum
    energy_d3 = cu_crystal_d3.get_potential_energy()
    current_step_results["PBE+DFTD3_Energy"] = energy_d3

    # --- PBE + DFT-D4 Calculation (using SumCalculator) ---
    cu_crystal_d4 = bulk('Cu', 'fcc', a=current_lc)
    gpaw_calc_d4_part = get_gpaw_calculator(f'Cu_D4_Sum_lc_{current_lc:.3f}.txt')
    dftd4_part = DFTD4(method="PBE")
    calc_d4_sum = SumCalculator([gpaw_calc_d4_part, dftd4_part])
    cu_crystal_d4.calc = calc_d4_sum
    energy_d4 = cu_crystal_d4.get_potential_energy()
    current_step_results["PBE+DFTD4_Energy"] = energy_d4
    
    all_results.append(current_step_results)

# --- Print All Results in a Table ---
print("\n" + "="*50)
print("             Overall Energy Summary            ")
print("="*50)
print("{:<10} {:<15} {:<15} {:<15}".format("LC (Å)", "PBE (eV)", "PBE+DFTD3 (eV)", "PBE+DFTD4 (eV)"))
print("-" * 55)
for res in all_results:
    print("{:<10.3f} {:<15.3f} {:<15.3f} {:<15.3f}".format(
        res["LC"], res["PBE_Energy"], res["PBE+DFTD3_Energy"], res["PBE+DFTD4_Energy"]
    ))
print("="*55)

print("\n--- Important Note ---")
print("Detailed SCF iteration output for each calculation is saved in separate '.txt' files.")
print("Please check the files named 'Cu_PBE_lc_*.txt', 'Cu_D3_Sum_lc_*.txt', and 'Cu_D4_Sum_lc_*.txt'")
print("to review the convergence behavior of each run.")

--- Starting Lattice Constant Scan (Detailed SCF output in .txt files) ---
Calculating for LC = 3.470 Å...
Calculating for LC = 3.525 Å...
Calculating for LC = 3.579 Å...
Calculating for LC = 3.633 Å...
Calculating for LC = 3.687 Å...

             Overall Energy Summary            
LC (Å)     PBE (eV)        PBE+DFTD3 (eV)  PBE+DFTD4 (eV) 
-------------------------------------------------------
3.470      -3.791          -4.363          -4.418         
3.525      -3.849          -4.398          -4.450         
3.579      -3.877          -4.403          -4.453         
3.633      -3.889          -4.394          -4.442         
3.687      -3.865          -4.350          -4.395         

--- Important Note ---
Detailed SCF iteration output for each calculation is saved in separate '.txt' files.
Please check the files named 'Cu_PBE_lc_*.txt', 'Cu_D3_Sum_lc_*.txt', and 'Cu_D4_Sum_lc_*.txt'
to review the convergence behavior of each run.
